# Residence Reccommender 
*CS524 Intro to Optimization, 18 December 2021*

Puqian Wang & Sonia Cromp

## Introduction
This is our project about apartments. Here is how we got the idea and why it's useful.

## Setup and Data Generation
We begin with preparing the "standard" dataset - more datasets will be used later in the analysis

In [1]:
import pandas as pd
%load_ext gams_magic 

### Resident data
Specify custom data parameters here

In [2]:
%%gams
set slot /t1*t5/;
# DO NOT EDIT ABOVE HERE
# number of residents
$set num_res 30
# stochastisity settings

# beds, baths distribution
# budget
# commute method and time

# DO NOT EDIT BELOW HERE
set r /1*%num_res%/;

In [3]:
%%gams
set labels /'nbed', 'nbath', 'budget', 'com_m', 'com_t'/;

parameter res(r, labels);

****************************************************
parameter rank_1(r, labels), rank(r, labels);

alias(labels, l);

parameter normalize(r);

loop(r,
rank_1(r, labels) = uniform(0,1);
normalize(r) = sum(labels, rank_1(r, labels));
);
rank(r, labels) = rank_1(r, labels)/normalize(r);

****************************************************

parameter beds;
loop(r,
    beds = normal(2,1);
    if(beds>0,
        res(r, 'nbed') = ceil(beds);
    else
        res(r, 'nbed') = 0;
    );
);

res(r, 'nbath') $ (res(r, 'nbed')>0)= floor(uniform(1, res(r, 'nbed')));
res(r, 'nbath') $ (res(r, 'nbed')=0) = 1;

parameter budgets;
loop(r,
    budgets = normal(1200,200);
    res(r, 'budget') = budgets $ (budgets>=500) + 500 $ (budgets<500);
);

*parameter com_m(r);
*com_m(r) = floor(uniform(1,4))

*$onText

res(r, 'com_m')  = floor(uniform(1,4));
res(r, 'com_m')  $ (res(r, 'com_m')=4) = 3

parameter time_slot(slot)
/t1 5,
 t2 10,
 t3 15,
 t4 20,
 t5 25/;

parameters com_time_prim, com_time;
loop(r,
com_time_prim = floor(normal(15, 5));
com_time $ (com_time_prim<time_slot('t1')) = 1;
com_time $ (com_time_prim>=time_slot('t1') and com_time_prim<time_slot('t2')) = 2;
com_time $ (com_time_prim>=time_slot('t2') and com_time_prim<time_slot('t3')) = 3;
com_time $ (com_time_prim>=time_slot('t3') and com_time_prim<time_slot('t4')) = 4;
com_time $ (com_time_prim>=time_slot('t5') and com_time_prim<time_slot('t5')) = 5;
com_time $ (com_time_prim>=time_slot('t5')) = 6;
res(r, 'com_t') = com_time;
)
*$offText

display res, rank, normalize;

execute_unload 'large_data' labels=headr, r = index, res=data;
execute_unload 'rank' labels=headr, r=index, rank=data;


In [4]:
%gams_pull -d res
%gams_pull -d rank

In [5]:
print(res.shape, rank.shape)
res.head()

(150, 3) (150, 3)


,r,labels,value
0,1,nbed,3.0
1,1,nbath,2.0
2,1,budget,1352.722697
3,1,com_m,1.0
4,1,com_t,4.0


In [6]:
rank.head()

,r,labels,value
0,1,nbed,0.079559
1,1,nbath,0.390629
2,1,budget,0.254952
3,1,com_m,0.139497
4,1,com_t,0.135362


In [7]:
# !wget https://www2.census.gov/geo/tiger/TIGER2019/COUSUB/tl_2019_55_cousub.zip
# !unzip tl_2019_55_cousub.zip

### Apartments data
Because gathering real apartments required some API calls where a credit card is required to sign up and will be charged upon exceeding the quota, this code is commented out and will not work without an API key. However, it is included so as to show how all the data was gathered.

First, gather the apartments themselves

In [8]:
# import requests
# url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"
# headers = {
#     'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
#     'x-rapidapi-key': "" # enter your api key here
#     }

# text = []
# for place in ['fitchburg', 'madison', 'middleton', 'sun prarie', 'verona']:
#     querystring = {"location":f"{place}, wi","home_type":"Condos,Apartments"}
#     response = requests.request("GET", url, headers=headers, params=querystring) # comes back in odd format that isn't quite JSON
#     text.append(response.text)

Second, determine the travel/commute time from each apartment to the CS building by walking, biking and driving

In [9]:
# import json
# import os
# import re

# allapts = []
# dest = '1210 W Dayton St Madison WI 53706'
# api_key = '' # enter your api key here
# url ='https://maps.googleapis.com/maps/api/distancematrix/json?'

# def gettime(source, dest, key, method='driving'):
#     source = re.sub('#', 'unit ', source)
#     # return response object
#     r = requests.get(url + 'origins=' + source +
#                     '&destinations=' + dest +
#                     '&mode=' + method +
#                     '&key=' + api_key)
#     # json method of response object
#     # return json format result
#     return r.json()

In [10]:
# for s in text:
#     # Convert from "almost" JSON to an actual JSON format
#     s = re.sub(f'\[?\d+ items', '', s)
#     s = re.sub(f'(?<=[L\"\de])\n(?!}})', ',\n', s)
#     s = re.sub(f'NULL', '-1', s)
#     s = re.sub(f'(\d+):', '\"\g<1>\":', s)
#     s = re.sub(f"\"props\":\n", '', s)
#     s = re.sub('}', '},', s)
#     s= re.sub(f'\n]\n', f'\n', s)
#     s = s.strip()[:-1] #since an extra comma is at the end

#     contents = json.loads(s)
#     contents.pop('resultsPerPage')
#     contents.pop('totalResultCount')
#     contents.pop('totalPages')

#     df = pd.DataFrame(contents).T
#     df.drop(['listingDateTime', 'listingStatus', 'zpid', 'lotAreaUnit', 'daysOnZillow', 'country', 'currency'], axis=1, inplace=True)

#     source = df['address'].to_list()

#     for m in ['driving', 'walking', 'bicycling']:
#         results = []
#         for i in range(len(source)//25): # API only can process 25 apartments at a time
#             subsource = source[25*i : 25*(i+1)]
#             print(len(subsource))
#             subsourcestr = '|'.join(subsource)
#             r = gettime(subsourcestr, dest, api_key, m)
#             results.append(r)

#         subsource = source[25*(len(source)//25) : ]
#         print(len(subsource))
#         subsourcestr = '|'.join(subsource)
#         r = gettime(subsourcestr, dest, api_key, m)
#         results.append(r)

#         seconds = []
#         for r in results:
#             for e in r['rows']:
#                 seconds.append(e['elements'][0]['duration']['value'])
#         df[m] = seconds
#         df[m] = (df[m]/60).astype(int)

#     df['price'] = df['price']/100
#     allapts.append(df)

# df = pd.concat(allapts)

# df.columns = ['nbath', 'propertyType', 'lotAreaValue', 'address', 'imgSrc', 'budget', 'longitude', 'latitude',
#                     'nbed', 'livingArea', 'hasImage', 'car', 'walk', 'bike']
# df.head()
# df.to_csv('apartments.csv', index=False) # save info

In [11]:
apartments = pd.read_csv('apartments.csv')
apt_data = apartments.drop(['propertyType', 'lotAreaValue', 'address', 'imgSrc', 'longitude', 'latitude',
                'livingArea', 'hasImage'], axis=1)
apt_data['a'] = apt_data.index+1
apt_data = apt_data[['a', 'nbed', 'nbath', 'budget', 'car', 'walk', 'bike']]
apt_data.columns = ['a', 'nbed', 'nbath', 'budget', 'm1', 'm2', 'm3']
# apt_data.drop(['m1', 'm2', 'm3'], axis=1, inplace=True)
apt_data['budget'] = apt_data['budget'].astype(int)
apt_data.set_index('a', inplace=True)
apt_data.head()

,nbed,nbath,budget,m1,m2,m3
a,,,,,,
1,2,2,1700,27,276,78
2,2,2,1980,29,301,83
3,2,2,1960,29,301,83
4,2,2,2100,27,299,88
5,1,1,1850,28,299,88


In [12]:
apt_data.to_csv('apartments_formatted.csv', index=False)

# Primary model
model and data description

residence data:
1. desired nbed, nbath, budget, com_m, com_t
2. nbed = 0 if the resident requires a studio

details:
1. com_t is an integer between 1-5, where 1-> t<5, 2-> 5<=t<10, ... 5-> t>25;
2. com_m is an integer between 1-3, where 1-> drive, 2-> walk, 3-> bike;
3. budget generated by normal distribution, mean=1200, delta=200;
4. extracted from gdx files: small_data, medium_data, large_data;
5. headr: /'nbed', 'nbath', 'budget', 'com_m', 'com_t'/
6. data: data(small/medium/large, headr);
7. small=30, medium=60, large=100;
8. need to figure out ranking -- currently a rand number form uniform(0,1) assigned to each label;

apartments data:
1. should have the same headr;
2. about commute methods: m1 for driving, m2 for walking, m3 for biking.

In [13]:
%%gams
set headr(*);
set r /1*30/;
set a /a1*a63/;

parameter pairs;
pairs = min(card(r), card(a));

$call csv2gdx apartments_formatted.csv id=Data autoRow=a values=1..lastCol useHeader=y

set apt_headr(*);
parameter apt_data(a, headr);


$gdxin apartments_new.gdx
$load headr=Dim2
$load apt_data=Data
$gdxin
display apt_data;


In [14]:
%%gams

**********************************************************
$onText
notes on the normalization of budgets/prices:

* for residents, since we know the budget is generated from norm(1200,200), we simply scale it by 600, hence the mean would be 2;
* for apartment prices, we scale it by 600 too.

$offtext
scalar budget_mean "the mean value of the budget, get it from residents_data" /1200/;
* normalize the budget with mean value 2
res(r, 'budget') = 2*res(r, 'budget')/budget_mean;

parameter aptprice(a);
apt_data(a,'budget') = 2*apt_data(a,'budget')/budget_mean;

# display apt_data;
***********************************************************

display res, rank;

set m "m1:drive, m2:walk, m3: drive" /m1*m3/;

parameter apt_com_t(a, m);

$gdxin apartments_formatted.gdx
$load apt_com_t=Data
$gdxin

display apt_data,apt_com_t;

********************** Format data **********************

set r_method(r,m) "include a (resident i, commute method j) pair in a set only if j is the preferred commute method of i";
r_method(r,m) = no;
r_method(r,m) $ (ord(m)=res(r,'com_m')) = yes;

parameter apt_com_t_nm(a,m) "binned commute time - put each commute time into 5-minute interval bins";

set slot /t1*t5/;
parameter time_slot(slot)
/t1 5,
 t2 10,
 t3 15,
 t4 20,
 t5 25/;

apt_com_t_nm(a,m) $ (apt_com_t(a, m)<time_slot('t1')) = 1;
apt_com_t_nm(a,m) $ (apt_com_t(a, m)>=time_slot('t1') and apt_com_t(a, m)<time_slot('t2')) = 2;
apt_com_t_nm(a,m) $ (apt_com_t(a, m)>=time_slot('t2') and apt_com_t(a, m)<time_slot('t3')) = 3;
apt_com_t_nm(a,m) $ (apt_com_t(a, m)>=time_slot('t3') and apt_com_t(a, m)<time_slot('t4')) = 4;
apt_com_t_nm(a,m) $ (apt_com_t(a, m)>=time_slot('t4') and apt_com_t(a, m)<time_slot('t5')) = 5;
apt_com_t_nm(a,m) $ (apt_com_t(a, m)>time_slot('t5')) = 6;

parameter
abs_diff(r,a,headr) "absolute difference between apartment a and and resident r",
abs_diff_cmt(r,a) "absolute difference of commute time between apartment a and resident r";

abs_diff(r,a,'nbed') = abs(apt_data(a, 'nbed')-res(r, 'nbed'));
abs_diff(r,a,'nbath') = abs(apt_data(a, 'nbath')-res(r, 'nbath'));
abs_diff(r,a,'budget') = abs(apt_data(a, 'budget')-res(r, 'budget'));
# abs_diff(r,a,'com_m') = res(r, 'com_m');
# abs_diff(r,a,'com_t') = res(r, 'com_t');
abs_diff_cmt(r,a) = abs(sum(m $ r_method(r,m), apt_com_t_nm(a,m)) -res(r,'com_t'));

display apt_data, abs_diff, abs_diff_cmt;

********************** Model **********************

free variable dis "dissatisfication";

variable rdis(r);

positive variables z(r,a,headr), zm(r,a);

Binary Variable
        b(r,a) "b(r,a)=1 if assign resident r to apartment i"
        d(r,m) "d(r, m)=1 if resident r choose method m";

equations
        obj, calcdis(r),
        bound_1(r) "each apartment gets at most one resident", 
        bound_2(a) "each resident gets at most one apartment", 
        bound_3 "count the number of residents assigned apartments";

obj..
dis =e= 1/card(r)* sum(r, rdis(r));

calcdis(r)..
rdis(r) =e= sum(a, b(r,a) * (rank(r, 'nbed')*abs_diff(r,a,'nbed') + rank(r, 'nbath')*abs_diff(r,a,'nbath') + rank(r, 'budget')*abs_diff(r,a,'budget')))
        + sum(a, b(r,a) * rank(r,'com_t') * abs_diff_cmt(r,a));
* second line is dissatisfaction with commute, first line is dissatisfaction with everything else

bound_1(r)..
sum(a, b(r,a)) =l= 1;

bound_2(a)..
sum(r, b(r,a)) =l= 1;

bound_3..
sum((r,a), b(r,a)) =e= pairs;

model primary_model /obj+calcdis+bound_1+bound_2+bound_3/;
solve primary_model using mip minimizing dis;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),0.5004,125,1921,MIP,CPLEX,0.04


Visualize, discuss

In [15]:
# %gams_lst

# Tradeoff model

*Varying lambda*

You will observe

Data scenarios

In [39]:
%%gams
* one abnormal resident
parameter rankold(labels);
rankold(labels) = rank('1')
rank('1', labels) = 0;
rank('1', 'nbed') = 1;
display rank;
solve tra_model using mip minimizing cdis;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),0.4908,2078,3813,MIP,CPLEX,0.248


In [40]:
%gams_lst

GAMS 36.2.0  r433180e Released Sep  3, 2021 DEX-DEG x86 64bit/Mac OS X - 12/17/21 02:41:19 Page 56
G e n e r a l   A l g e b r a i c   M o d e l i n g   S y s t e m
C o m p i l a t i o n


 427  * one abnormal resident
 428  rank('1', labels) = 0;
 429  rank('1', 'nbed') = 1;
 430  display rank;
 431  solve tra_model using mip minimizing cdis;


COMPILATION TIME     =        0.000 SECONDS      4 MB  36.2.0 r433180e DEX-DEG
GAMS 36.2.0  r433180e Released Sep  3, 2021 DEX-DEG x86 64bit/Mac OS X - 12/17/21 02:41:19 Page 57
G e n e r a l   A l g e b r a i c   M o d e l i n g   S y s t e m
E x e c u t i o n


----    430 PARAMETER rank  

          nbed       nbath      budget       com_m       com_t

1        1.000
2        0.112       0.175       0.429       0.034       0.250
3        0.288       0.167       0.286       0.220       0.038
4        0.297       0.074       0.116       0.311       0.202
5        0.227       0.222       0.083       0.095       0.372
6        0.296       0.082 

# Stochastic tradeoff model
stochastic model details:

* match every resident registered in the past week to current apartments every Mon;
* each resident is assigned a register time, from 1-7, where 1 -> 1 day ago, 2 -> 2 days ago, etc.;
* for each register time, there is a pobability that the resident would quit before the matching process;
* this probability is set manually;
* if a resident quits, his/hers dissatisfication is set to 0; 
* the rest of the model remain unchanged;

In [31]:
%%gams
set t week days /1*7/;

parameter regtim(r) "the registration time of each resident r";

regtim(r) = ceil(uniform(0,7));

parameter quitprob(t) "the probability of quitting on the t-th days after registration"
/1 0.05
 2 0.05
 3 0.7
 4 0.9
 5 0.12
 6 0.15
 7 0.18/;

equation calcErdis(r,t) "calculate the expected dissatisfication of resident r";

calcErdis(r,t) $ (ord(t)=regtim(r))..
rdis(r) =e= (1-quitprob(t))*sum(a, b(r,a) * (rank(r, 'nbed')*abs_diff(r,a,'nbed') + rank(r, 'nbath')*abs_diff(r,a,'nbath') + rank(r, 'budget')*abs_diff(r,a,'budget')))
        + (1-quitprob(t))*sum(a, b(r,a) * rank(r,'com_t')*abs_diff_cmt(r,a));
* second line is Expected dissatisfaction with commute, first line is expected dissatisfaction with everything else

model stotra_model /obj, calcavgdis, calcmaxdis, calcErdis, findbetter, findworse, calcIsntmax,
                calcIsmax, mostdissed, disaon_lolim1, disaon_lolim2, disaon_hilim, bound_1, 
                bound_2, bound_3/;
solve stotra_model using mip min cdis;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),0.7322,2078,3813,MIP,CPLEX,1.739


In [28]:
%gams_lst

GAMS 36.2.0  r433180e Released Sep  3, 2021 DEX-DEG x86 64bit/Mac OS X - 12/17/21 02:30:06 Page 40
G e n e r a l   A l g e b r a i c   M o d e l i n g   S y s t e m
C o m p i l a t i o n


 318  set t week days /1*7/;
 319   
 320  parameter regtim(r) "the registration time of each resident r";
 321   
 322  regtim(r) = ceil(uniform(0,7));
 323   
 324  parameter quitprob(t) "the probability of quitting on the t-th days after registration"
 325  /1 0.05
 326   2 0.05
 327   3 0.7
 328   4 0.9
 329   5 0.12
 330   6 0.15
 331   7 0.18/;
 332   
 333  equation calcErdis(r,t) "calculate the expected dissatisfication of resident r";
 334   
 335  calcErdis(r,t) $ (ord(t)=regtim(r))..
 336  rdis(r) =e= (1-quitprob(t))*sum(a, b(r,a) * (rank(r, 'nbed')*abs_diff(r,a,'nbed') + rank(r, 'nbath')*abs_diff(r,a,'nbath') + rank(r, 'budget')*abs_diff(r,a,'budget')))
 337          + (1-quitprob(t))*sum(a, b(r,a) * rank(r,'com_t')*abs_diff_cmt(r,a));
 338  * second line is Expected dissatisfaction with 